In [1]:
import itertools
import operator
import re
from abc import *
from copy import deepcopy
from operator import itemgetter
from typing import *
from nodestimation.learning.connectoming import *
from nodestimation.learning.modification import normalize_df
import numpy as np
import scipy as sp
import networkx as nx
import mne
import time
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors._dist_metrics import DistanceMetric
from sklearn.utils import shuffle
from typing import List, Tuple
from sklearn.model_selection import train_test_split
from nodestimation.learning.estimation import collect_statistic, \
    compute_importance, collect_cross_statistic, make_selection_map, \
    select, separate_datasets, selected_statistic, choose_best, selected_data, make_feature_selection
from nodestimation.learning.informativeness import CrossInformativeness, Informativeness, SubjectsInformativeness, \
    NodesInformativeness
from nodestimation.learning.networking import sparse_graph, graph_to_hemispheres, hemispheres_division_modularity, \
    metric_for_hemispheres
from nodestimation.processing.features import prepare_features
from nodestimation.project import find_subject_dir, conditions_unique_code
from nodestimation.pipeline import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import nibabel
import nilearn.plotting as nplt
from nodestimation.project.actions import read
import nodestimation as nd
from nodestimation.learning.modification import append_series, promote
import nodestimation.learning.modification as lmd
from nodestimation.project.subject import Subject
from sklearn.preprocessing import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
import matplotlib as mpl
from nodestimation.learning.selection import SubjectsStatistic, Wilcoxon, Mannwhitneyu, Test
from scipy.stats import wilcoxon
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from scipy.fftpack import fftfreq, irfft, rfft
from scipy.fftpack import fftfreq, irfft, rfft

In [3]:

SUBJECTS_ORIGINAL = pipeline(
    methods=['wpli', 'envelope', 'coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli2_debiased'],
    freq_bands=(4, 8),
    centrality_metrics=['eigen', 'between', 'degree', 'info']
    )

ENGEL1 = [
    'B1C2',
    'B1R1',
    'G1R1',
    'G1V2',
    'J1T2',
    'K1V1',
    'L1P1',
    'M1G2',
    'M1N2',
    'O1O2',
    'R1D2',
    'S1A2',
    'S1B1',
    'S1H1',
    'S1U3'
]
ENGEL2 = [
    'L2M1',
    'M2S2',
    'N2K2',
    'P1H2'
]
ENGEL34 = [
    'N3S2',
    'S3R1',
    'K4L2'
]
REJECTED = [
    'S1U3',
    'P1H2'
]

SUBJECTS = [
    subject
    for subject in SUBJECTS_ORIGINAL
    if subject.name not in ENGEL34
       and subject.name not in REJECTED
]

CONNECTOMES_KINDS = [
    'initial',
    # 'binary',
    # 'suppressed',
    'inverse',
    # 'inverse_binary',
    # 'inverse_suppressed',
    # 'spatial',
    'initial&spatial',
    'inverse&spatial',
    # 'bin&spatial',
    # 'supp&spatial',
    # 'inverse-bin&spatial',
    # 'inverse-supp&spatial'

]

series = list()
index = list()

for CONNECTOMES_KIND in CONNECTOMES_KINDS:
    print(f'-------------------------------------------------------\n'
          f'{CONNECTOMES_KIND.capitalize()} connectomes computation\n'
          f'-------------------------------------------------------\n')

    subjects = SUBJECTS.copy()

    DATASETS = {
        subject.name: {
            freq: {
                method: make_connectome(subject, freq, method, CONNECTOMES_KIND, threshold=1)
               for method in subject.connectomes[freq]
            }
            for freq in subject.connectomes
        }
        for subject in subjects
    }
    print('Connectomes are prepared')

    datasets = DATASETS.copy()

    IGNORED_METHODS = [
        # 'envelope',
        # 'wpli',
        'coh',
        'imcoh',
    #     'plv',
        'ciplv',
        'ppc',
        'pli',
        'pli2_unbiased',
        'wpli2_debiased'
    ]

    GRAPHS = [
        metric_for_hemispheres(subjects, nx.algorithms.cluster.transitivity),
        metric_for_hemispheres(subjects, nx.algorithms.smetric.s_metric, normalized=False),
        metric_for_hemispheres(subjects, nx.algorithms.global_efficiency),
    ]

    for graph, name in zip(
            GRAPHS,
            [
                'transitivity',
                's_metric',
                'global_efficiency'
            ]
    ):
        for features in [['wpli', 'envelope'], ['plv']]:
            acc, spec, sens, pospred, negpred = list(), list(), list(), list(), list()
            for _ in range(1000):
                y = graph['resected'].to_numpy()
                x = graph[[
                    f'{name}_for_{feat}_4-8Hz'
                    for feat in features
                ]].to_numpy()
                scaler = StandardScaler()
                x = scaler.fit_transform(x)
                x_train, x_test, y_train, y_test = train_test_split(x, y)

                # clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=10)
                # clf = AdaBoostClassifier(n_estimators=10)
                clf = svm.SVC(kernel='sigmoid')
                # clf = svm.SVC()
                # clf = svm.SVC(kernel='linear')
                # clf = svm.SVC(kernel='poly')
                # clf = SGDClassifier()
                # clf = KNeighborsClassifier(n_neighbors=7, metric='chebyshev')
                # clf = LogisticRegression(class_weight={True: 1, False: .8})
                # clf = LogisticRegression()
                # clf = RandomForestClassifier(max_depth=20)
                # clf = GaussianNB()
                # clf = LinearDiscriminantAnalysis()
                # clf = QuadraticDiscriminantAnalysis()
                # clf = KMeans(n_clusters=2, algorithm='full')
                # clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(10, 10), max_iter=1450)

                clf.fit(x_train, y_train)
                pred = clf.predict(x_test)
                acc.append(accuracy_score(y_test, pred))
                tn, fp, fn, tp = confusion_matrix(y_test, pred, labels=[0,1]).ravel()
                loc_spec = tn / (tn + fp)
                loc_sens = tp / (tp + fn)

                if not np.isnan(loc_spec):
                    spec.append(loc_spec)
                if not np.isnan(loc_sens):
                    sens.append(loc_sens)

            # if np.isnan(tp/(tp + fp)):
            #     pospred.append(0)
            # else:
            #     pospred.append(tp/(tp + fp))
            #
            # if np.isnan((tn/(tn + fn))):
            #     negpred.append(0)
            # else:
            #     negpred.append(tn/(tn + fn))

        # print(f'CONNECTOMES_KIND: {CONNECTOMES_KIND}')
        # print(f'name: {name}, features: {features}')
        # print('acc: ', np.array(acc).mean())
        # print('spec: ', np.array(spec).mean())
        # print('sens: ', np.array(sens).mean())

            series.append(
                {
                    'acc': np.array(acc).mean(),
                    'spec': np.array(spec).mean(),
                    'sens': np.array(sens).mean()
                }
            )
            index.append(
                f'{CONNECTOMES_KIND}+'
                f'{name}+'
                f'{features}'
            )


final = pd.DataFrame(series, index=index)

final.to_csv(
    '/home/user/Documents/Hemispheres-spatial-classifier-performance.csv'
)

# final.to_csv(
#     '/home/user/Documents/Basic-spatial-classifier-performance-Engel1&2&no-rejected.csv'
# )

All computation has been already done, loading of the existing file with the solution...
-------------------------------------------------------
Initial connectomes computation
-------------------------------------------------------

Connectomes are prepared
M2S2: DONE, RUNTIME: 9.215765476226807
R1D2: DONE, RUNTIME: 10.290708303451538
S1A2: DONE, RUNTIME: 7.766717195510864
S1H1: DONE, RUNTIME: 10.877424240112305
K1V1: DONE, RUNTIME: 10.572423934936523
L1P1: DONE, RUNTIME: 8.37821340560913
M1G2: DONE, RUNTIME: 10.321546792984009
G1V2: DONE, RUNTIME: 7.662637948989868
G1R1: DONE, RUNTIME: 7.707006216049194
M1N2: DONE, RUNTIME: 8.533468961715698
S1B1: DONE, RUNTIME: 9.179703950881958
B1R1: DONE, RUNTIME: 7.7054619789123535
N2K2: DONE, RUNTIME: 14.534250974655151
B1C2: DONE, RUNTIME: 9.447551012039185
J1T2: DONE, RUNTIME: 9.000712156295776
O1O2: DONE, RUNTIME: 9.178578615188599
L2M1: DONE, RUNTIME: 10.437699556350708
M2S2: DONE, RUNTIME: 5.826844930648804
R1D2: DONE, RUNTIME: 6.6060547828

<ipython-input-3-b13e27547283>:148: RuntimeWarning: invalid value encountered in long_scalars
  loc_spec = tn / (tn + fp)
<ipython-input-3-b13e27547283>:148: RuntimeWarning: invalid value encountered in long_scalars
  loc_spec = tn / (tn + fp)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:148: RuntimeWarning: invalid value encountered in long_scalars
  loc_spec = tn / (tn + fp)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)


-------------------------------------------------------
Inverse connectomes computation
-------------------------------------------------------

Connectomes are prepared
M2S2: DONE, RUNTIME: 9.459542751312256
R1D2: DONE, RUNTIME: 10.524528980255127
S1A2: DONE, RUNTIME: 7.883581638336182
S1H1: DONE, RUNTIME: 11.152165174484253
K1V1: DONE, RUNTIME: 10.557624816894531
L1P1: DONE, RUNTIME: 8.408409595489502
M1G2: DONE, RUNTIME: 10.417073488235474
G1V2: DONE, RUNTIME: 7.820811748504639
G1R1: DONE, RUNTIME: 7.76660943031311
M1N2: DONE, RUNTIME: 8.514785289764404
S1B1: DONE, RUNTIME: 9.221323013305664
B1R1: DONE, RUNTIME: 7.728970766067505
N2K2: DONE, RUNTIME: 14.808879613876343
B1C2: DONE, RUNTIME: 9.729835271835327
J1T2: DONE, RUNTIME: 9.155750036239624
O1O2: DONE, RUNTIME: 9.324788331985474
L2M1: DONE, RUNTIME: 10.83662748336792
M2S2: DONE, RUNTIME: 5.92214035987854
R1D2: DONE, RUNTIME: 6.547825574874878
S1A2: DONE, RUNTIME: 5.132509708404541
S1H1: DONE, RUNTIME: 6.941655158996582
K1V1: DO

<ipython-input-3-b13e27547283>:148: RuntimeWarning: invalid value encountered in long_scalars
  loc_spec = tn / (tn + fp)
<ipython-input-3-b13e27547283>:148: RuntimeWarning: invalid value encountered in long_scalars
  loc_spec = tn / (tn + fp)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)


-------------------------------------------------------
Initial&spatial connectomes computation
-------------------------------------------------------

Connectomes are prepared
M2S2: DONE, RUNTIME: 9.286571979522705
R1D2: DONE, RUNTIME: 10.381111145019531
S1A2: DONE, RUNTIME: 7.757688999176025
S1H1: DONE, RUNTIME: 10.95764684677124
K1V1: DONE, RUNTIME: 10.448082447052002
L1P1: DONE, RUNTIME: 8.262898445129395
M1G2: DONE, RUNTIME: 10.2926504611969
G1V2: DONE, RUNTIME: 7.760442018508911
G1R1: DONE, RUNTIME: 8.167979001998901
M1N2: DONE, RUNTIME: 8.615393877029419
S1B1: DONE, RUNTIME: 9.22554349899292
B1R1: DONE, RUNTIME: 7.712081670761108
N2K2: DONE, RUNTIME: 14.55559754371643
B1C2: DONE, RUNTIME: 9.456129550933838
J1T2: DONE, RUNTIME: 9.027268409729004
O1O2: DONE, RUNTIME: 9.237074375152588
L2M1: DONE, RUNTIME: 10.929757595062256
M2S2: DONE, RUNTIME: 6.069267272949219
R1D2: DONE, RUNTIME: 6.409238576889038
S1A2: DONE, RUNTIME: 4.990667104721069
S1H1: DONE, RUNTIME: 6.637614965438843
K1

<ipython-input-3-b13e27547283>:148: RuntimeWarning: invalid value encountered in long_scalars
  loc_spec = tn / (tn + fp)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:148: RuntimeWarning: invalid value encountered in long_scalars
  loc_spec = tn / (tn + fp)
<ipython-input-3-b13e27547283>:148: RuntimeWarning: invalid value encountered in long_scalars
  loc_spec = tn / (tn + fp)


-------------------------------------------------------
Inverse&spatial connectomes computation
-------------------------------------------------------

Connectomes are prepared
M2S2: DONE, RUNTIME: 9.189542055130005
R1D2: DONE, RUNTIME: 10.35629653930664
S1A2: DONE, RUNTIME: 7.719861745834351
S1H1: DONE, RUNTIME: 10.944894790649414
K1V1: DONE, RUNTIME: 10.384186029434204
L1P1: DONE, RUNTIME: 8.212997674942017
M1G2: DONE, RUNTIME: 10.282302856445312
G1V2: DONE, RUNTIME: 7.65546178817749
G1R1: DONE, RUNTIME: 7.635318994522095
M1N2: DONE, RUNTIME: 8.428850889205933
S1B1: DONE, RUNTIME: 9.159467220306396
B1R1: DONE, RUNTIME: 7.662721872329712
N2K2: DONE, RUNTIME: 14.568693161010742
B1C2: DONE, RUNTIME: 9.378011226654053
J1T2: DONE, RUNTIME: 9.021826982498169
O1O2: DONE, RUNTIME: 9.283644199371338
L2M1: DONE, RUNTIME: 10.685567617416382
M2S2: DONE, RUNTIME: 5.875538110733032
R1D2: DONE, RUNTIME: 6.557312250137329
S1A2: DONE, RUNTIME: 5.032425403594971
S1H1: DONE, RUNTIME: 6.738199472427368

<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:148: RuntimeWarning: invalid value encountered in long_scalars
  loc_spec = tn / (tn + fp)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:149: RuntimeWarning: invalid value encountered in long_scalars
  loc_sens = tp / (tp + fn)
<ipython-input-3-b13e27547283>:148: RuntimeWarning: invalid value encountered in long_scalars
  loc_spec = tn / (tn + fp)
